# REPORT 1 - Economic considerations of different boosting priority strategies

In this notebook we compute the economic burdens of different age-priority boosting strategies, using the `warwickmodel`, built by Universities of Warwick and Lancaster, with population data from 8 countries with very different socio-economic profiles.

The effectiveness cost of boosters is quantified in terms of reductions in total amount of years of life lost due to boosyer vaccines deploymnet. We assume an initial boosting campaign in the population, with no subsequent boosters being deployed during the simulation. 

The infection dynamics are run for:
 - Dates: **15 Feb 2020** - **25 June 2021**;
 - Countries of interest: **United Kingdom**, **France**;
 - Number of boosters deployed: **10%** of the population.

*The Warwick model is built by Universities of Warwick and Lancaster.*

In [1]:
# Load necessary libraries
import os
import copy
import numpy as np
import pandas as pd
import scipy
import epimodels as em
import warwickmodel as wm
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the WarwickLanc Model

In [2]:
# Populate the model
total_days =  100
regions = ['United Kingdom', 'Canada', 'Brazil', 'South Africa', 'Kenya', 'Philippines', 'Sierra Leone', 'Syria']
age_groups = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
              '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75+']

regimes = np.arange(1, 101, 1).tolist()

# Add folder path to data file
path = os.path.join('../data/')

### Read in corresponding data files for the countries considered for all 100 regimes

In [3]:
# Matrices contact
matrices_contact = []
matrices_region = []
time_changes_contact = []
time_changes_region = []

# Vaccine effects
nu_tra = []
nu_symp = []
nu_inf = []
nu_sev_h = []
nu_sev_d = []

# Parameters
omega = []
alpha = []
gamma = []
tau = []
we = []

# Initial conditions
susceptibles_IC = []
exposed1_IC = []
exposed2_IC = []
exposed3_IC = []
exposed4_IC = []
exposed5_IC = []
infectives_sym_IC = []
infectives_asym_IC = []
recovered_IC = []

# Risk factors
d = []
beta = []

# Probabilities of proceeding to severe outcomes
pItoH = []
pHtoD = []

# Distribution of delays before proceeding to severe outcomes
dItoH = []
dHtoD = []

for R in regimes:
        regimes_matrices_region = []

        # Initial state of the system
        weeks_matrices_region = []
        for r in regions:
                region_data_matrix = pd.read_csv(
                        os.path.join(path, '{}/Contacts_{}.csv'.format(r, R)),
                        header=None, dtype=np.float64)
                regional = em.RegionMatrix(r, age_groups, region_data_matrix)
                weeks_matrices_region.append(regional)

        regimes_matrices_region.append(weeks_matrices_region)

        contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
        regimes_matrices_contact = [contacts]

        matrices_region.append(regimes_matrices_region)
        matrices_contact.append(regimes_matrices_contact)

        # Matrices contact
        time_changes_contact.append([1])
        time_changes_region.append([1])

        # Over 75 population fractions
        frac_pop_over75 = []

        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        skiprows=15,
                        header=None, dtype=np.float64)

                frac_pop_over75.append((1/np.sum(np.asarray(IC_df))) * np.sum(np.asarray(IC_df),axis=1))

        # Risk Factors
        extended_regimes_d = []
        regimes_d = []
        regimes_beta = []

        for r, reg in enumerate(regions):
                RF_df = pd.read_csv(
                        os.path.join(path, '{}/Risks_{}.csv'.format(reg, R)),
                        dtype=np.float64)
                extended_d = RF_df['symptom_risk'].tolist()
                extended_beta = RF_df['susceptibility'].tolist()

                extended_regimes_d.append(extended_d)

                regimes_d.append(extended_d[:15] + [np.sum(np.multiply(extended_d[15:], frac_pop_over75[r]))])
                regimes_beta.append(extended_beta[:15] + [np.sum(np.multiply(extended_beta[15:], frac_pop_over75[r]))])

        d.append(regimes_d)
        beta.append(regimes_beta)

        # Vaccine effects
        eff_df = pd.read_csv(
                os.path.join(path, 'global_parameters/efficacies_{}.csv'.format(R)),
                usecols=range(1,5), dtype=np.float64)

        VE_i = eff_df['Infection_eff']
        VE_s = eff_df['Symptom_eff']
        VE_h = eff_df['Hosp_eff']
        VE_d = eff_df['Death_eff']

        VE_d = np.divide(VE_d-VE_h, 1-VE_h)
        VE_h = np.divide(VE_h-VE_i, 1-VE_i)
        VE_s = np.divide(VE_s-VE_i, 1-VE_i)

        regimes_nu_tra = [1] * 6
        regimes_nu_symp = np.nan_to_num(1 - VE_s).tolist()
        regimes_nu_inf = np.nan_to_num(1 - VE_i).tolist()
        regimes_nu_sev_h = np.nan_to_num(1 - VE_h).tolist()
        regimes_nu_sev_d = np.nan_to_num(1 - VE_d).tolist()

        nu_tra.append(regimes_nu_tra)
        nu_symp.append(regimes_nu_symp)
        nu_inf.append(regimes_nu_inf)
        nu_sev_h.append(regimes_nu_sev_h)
        nu_sev_d.append(regimes_nu_sev_d)

        # Parameters
        param_df = pd.read_csv(
                os.path.join(path, 'global_parameters/parameters_{}.csv'.format(R)),
                dtype=np.float64)

        regimes_omega = param_df['transmission'].tolist()[0]
        regimes_alpha = 1
        regimes_gamma = param_df['recovery'].tolist()[0]
        regimes_tau = param_df['asymptomatic_transmission'].tolist()[0]
        regimes_we = [param_df['waning_rate'].tolist()[0]] * 2 + [0]

        omega.append(regimes_omega)
        alpha.append(regimes_alpha)
        gamma.append(regimes_gamma)
        tau.append(regimes_tau)
        we.append(regimes_we)

        # Initial conditions
        regimes_susceptibles_IC = []
        regimes_exposed1_IC = []
        regimes_exposed2_IC = []
        regimes_exposed3_IC = []
        regimes_exposed4_IC = []
        regimes_exposed5_IC = []
        regimes_infectives_sym_IC = []
        regimes_infectives_asym_IC = []
        regimes_recovered_IC = []

        # Susceptible
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(0, 5),
                        header=None, dtype=np.float64)

                extended_S = np.asarray(IC_df)
                under_75_S = extended_S[:15, :]
                over_75_S = extended_S[15:, :]
                reduced_S = np.vstack((under_75_S, np.sum(over_75_S, axis=0)))
                regimes_susceptibles_IC.append(
                        reduced_S.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 1
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(5, 10),
                        header=None, dtype=np.float64)

                extended_E1 = np.asarray(IC_df)
                under_75_E1 = extended_E1[:15, :]
                over_75_E1 = extended_E1[15:, :]
                reduced_E1 = np.vstack((under_75_E1, np.sum(over_75_E1, axis=0)))
                regimes_exposed1_IC.append(
                        reduced_E1.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 2
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(10, 15),
                        header=None, dtype=np.float64)

                extended_E2 = np.asarray(IC_df)
                under_75_E2 = extended_E2[:15, :]
                over_75_E2 = extended_E2[15:, :]
                reduced_E2 = np.vstack((under_75_E2, np.sum(over_75_E2, axis=0)))
                regimes_exposed2_IC.append(
                        reduced_E2.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 3
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(15, 20),
                        header=None, dtype=np.float64)

                extended_E3 = np.asarray(IC_df)
                under_75_E3 = extended_E3[:15, :]
                over_75_E3 = extended_E3[15:, :]
                reduced_E3 = np.vstack((under_75_E3, np.sum(over_75_E3, axis=0)))
                regimes_exposed3_IC.append(
                        reduced_E3.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 4
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(20, 25),
                        header=None, dtype=np.float64)

                extended_E4 = np.asarray(IC_df)
                under_75_E4 = extended_E4[:15, :]
                over_75_E4 = extended_E4[15:, :]
                reduced_E4 = np.vstack((under_75_E4, np.sum(over_75_E4, axis=0)))
                regimes_exposed4_IC.append(
                        reduced_E4.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 5
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(25, 30),
                        header=None, dtype=np.float64)

                extended_E5 = np.asarray(IC_df)
                under_75_E5 = extended_E5[:15, :]
                over_75_E5 = extended_E5[15:, :]
                reduced_E5 = np.vstack((under_75_E5, np.sum(over_75_E5, axis=0)))
                regimes_exposed5_IC.append(
                        reduced_E5.flatten('F').tolist() + [0] * len(age_groups))

        # Symptomatic & Asymptomatic Infectious
        for _, r in enumerate(regions):
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(30, 35),
                        header=None, dtype=np.float64)

                extended_I = np.zeros_like(np.asarray(IC_df))
                extended_I[:, 0] = np.matmul(np.diag(regimes_nu_symp[0] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 0])
                extended_I[:, 1] = np.matmul(np.diag(regimes_nu_symp[1] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 1])
                extended_I[:, 2] = np.matmul(np.diag(regimes_nu_symp[2] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 2])
                extended_I[:, 3] = np.matmul(np.diag(regimes_nu_symp[3] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 3])
                extended_I[:, 4] = np.matmul(np.diag(regimes_nu_symp[4] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 4])
                under_75_I = extended_I[:15, :]
                over_75_I = extended_I[15:, :]
                reduced_I = np.vstack((under_75_I, np.sum(over_75_I, axis=0)))
                regimes_infectives_sym_IC.append(
                        reduced_I.flatten('F').tolist() + [0] * len(age_groups))

                extended_A = np.zeros_like(np.asarray(IC_df))
                extended_A[:, 0] = np.matmul(np.diag((1 - regimes_nu_symp[0] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 0])
                extended_A[:, 1] = np.matmul(np.diag((1 - regimes_nu_symp[1] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 1])
                extended_A[:, 2] = np.matmul(np.diag((1 - regimes_nu_symp[2] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 2])
                extended_A[:, 3] = np.matmul(np.diag((1 - regimes_nu_symp[3] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 3])
                extended_A[:, 4] = np.matmul(np.diag((1 - regimes_nu_symp[4] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 4])
                under_75_A = extended_A[:15, :]
                over_75_A = extended_A[15:, :]
                reduced_A = np.vstack((under_75_A, np.sum(over_75_A, axis=0)))
                regimes_infectives_asym_IC.append(
                        reduced_A.flatten('F').tolist() + [0] * len(age_groups))

        # Recovered
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=[35],
                        header=None, dtype=np.float64)

                extended_R = np.asarray(IC_df)
                under_75_R = extended_R[:15, :]
                over_75_R = extended_R[15:, :]
                reduced_R = np.vstack((under_75_R, np.sum(over_75_R, axis=0)))
                regimes_recovered_IC.append(
                        reduced_R.flatten('F').tolist())

        susceptibles_IC.append(regimes_susceptibles_IC)
        exposed1_IC.append(regimes_exposed1_IC)
        exposed2_IC.append(regimes_exposed2_IC)
        exposed3_IC.append(regimes_exposed3_IC)
        exposed4_IC.append(regimes_exposed4_IC)
        exposed5_IC.append(regimes_exposed5_IC)
        infectives_sym_IC.append(regimes_infectives_sym_IC)
        infectives_asym_IC.append(regimes_infectives_asym_IC)
        recovered_IC.append(regimes_recovered_IC)

        # Set time-to-hospitalisation using a Gamma distribution using the mean and standard deviation 
        th_mean = param_df['hosp_lag'].tolist()[0]+0.00001
        th_var = 12.1**2
        theta = th_var / th_mean
        k = th_mean / theta
        time_to_hosp = scipy.stats.gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

        # Set time-to-death using a Gamma distribution using the mean and standard deviation
        td_mean = param_df['death_lag'].tolist()[0]
        td_var = 12.1**2
        theta = td_var / td_mean
        k = td_mean / theta
        time_to_death = scipy.stats.gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

        # Probabilities of proceeding to severe outcomes
        # Infected -> Hospital
        extended_pItoH = RF_df['hospitalisation_risk'].tolist()

        regimes_pItoH = []
        for r, reg in enumerate(regions):
                regimes_pItoH.append(extended_pItoH[:15] + [np.sum(np.multiply(extended_pItoH[15:], frac_pop_over75[r]))])

        pItoH.append(regimes_pItoH)

        # Hospital -> Death
        extended_pHtoD = RF_df['death_risk'].tolist()

        regimes_pHtoD = []
        for r, reg in enumerate(regions):
                regimes_pHtoD.append(extended_pHtoD[:15] + [np.sum(np.multiply(extended_pHtoD[15:], frac_pop_over75[r]))])

        pHtoD.append(regimes_pHtoD)

        # Distribution of delays before proceeding to severe outcomes
        # Infected -> Hospital
        dItoH.append(time_to_hosp)
        # Hospital -> Death
        dHtoD.append(time_to_death)

# Other parameters
vac=0
vacb=0

adult = np.ones(len(age_groups))
adult[0] = 0
adult[1] = 0
adult[2] = 0

### Calculate life expectancy for each region

In [4]:
# Read region-dependent vectors of years of life lost
life_ex = []

for r, reg in enumerate(regions):
        # Read life expectancy data
        LE_df = pd.read_csv(os.path.join(path, 'Life_expectancy_{}.csv'.format(reg)),
                usecols=[1, 2], dtype=np.float64)
        LE = np.array(LE_df)

        # Read age & sex distribution data
        ASD_df = pd.read_csv(os.path.join(path, 'Age_sex_distribution_{}.csv'.format(reg)),
                usecols=[1, 2], dtype=np.float64)
        extended_ASD = np.array(ASD_df)
        ASD = np.zeros_like(LE)

        # Group the ages of the ASD into the same age groups as the LE
        sum_ages = [
                0, range(1, 5), range(5, 10), range(10, 15), range(15, 20), range(20, 25), range(25, 30), range(30, 35), range(35, 40), 
                range(40, 45), range(45, 50), range(50, 55), range(55, 60), range(60, 65), range(65, 70), range(70, 75), range(75, 80), 
                range(80, 85), range(85, 101)]

        ASD[0, :] = extended_ASD[sum_ages[0], :]

        for _, ages in enumerate(sum_ages[1:]):
                ASD[_ + 1, :] = np.sum(extended_ASD[ages, :], axis=0)

        frac_ASD = np.matmul(np.diag(1/np.sum(ASD, axis=1)), ASD)

        # Compute the life expectancy of the different age groups
        total_LE = np.sum(np.multiply(frac_ASD, LE), axis=1)

        # Read Life expectancy data
        frac_AD = (1/np.sum(ASD[:2, :]) * np.sum(ASD[:2, :], axis=1)).tolist() + [1] * 14 + (1/np.sum(ASD[16:, :]) * np.sum(ASD[16:, :], axis=1)).tolist()

        # Compute the life expectancy of the correct age groups
        split_LE = np.multiply(frac_AD, total_LE)
        
        final_LE = np.zeros(len(age_groups))
        final_LE[0] = np.sum(split_LE[:2])
        final_LE[1:15] = split_LE[2:16]
        final_LE[15] = np.sum(split_LE[16:]).tolist()

        life_ex.append(final_LE)
      

### Calculate total population for each regime

In [5]:
# Compute the number of boosters for each region
total_pop = []

for R in regimes:
        regimes_total_pop = []
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        header=None, dtype=np.float64)

                regimes_total_pop.append(np.sum(np.asarray(IC_df)))
        total_pop.append(regimes_total_pop)

## Boosting campaign scenarios

We only boost the partially-waned and recovered. However those in the R compartment who receive the booster do not move out of the compartment (they have higher immunity than the boosted)

In [6]:
# Maximum percentage booster uptake of each age group
boost_age_percent = 0.9

# Compute the maximum number of boosters we can deploy for each age group in each region
max_boosters = []
max_boosters_for_R = []
max_boosters_for_S = []
max_boosters_for_Sf = []
max_boosters_for_Sw1 = []
max_boosters_for_Sw2 = []
max_boosters_for_Sw3 = []

for R, _ in enumerate(regimes):
        regimes_max_boosters = []
        regimes_max_boosters_for_R = []
        regimes_max_boosters_for_S = []
        regimes_max_boosters_for_Sf = []
        regimes_max_boosters_for_Sw1 = []
        regimes_max_boosters_for_Sw2 = []
        regimes_max_boosters_for_Sw3 = []

        for r, reg in enumerate(regions):
                boosters_for_R = boost_age_percent * np.asarray(recovered_IC[R][r])
                boosters_for_S = boost_age_percent * np.asarray(susceptibles_IC[R][r])[:len(age_groups)]
                boosters_for_Sf = boost_age_percent * np.asarray(susceptibles_IC[R][r])[len(age_groups):(2*len(age_groups))]
                boosters_for_Sw1 = boost_age_percent * np.asarray(susceptibles_IC[R][r])[(3*len(age_groups)):(4*len(age_groups))]
                boosters_for_Sw2 = boost_age_percent * np.asarray(susceptibles_IC[R][r])[(4*len(age_groups)):(5*len(age_groups))]
                boosters_for_Sw3 = boost_age_percent * np.asarray(susceptibles_IC[R][r])[(5*len(age_groups)):(6*len(age_groups))]

                regimes_max_boosters_for_R.append(boosters_for_R)
                regimes_max_boosters_for_S.append(boosters_for_S)
                regimes_max_boosters_for_Sf.append(boosters_for_Sf)
                regimes_max_boosters_for_Sw1.append(boosters_for_Sw1)
                regimes_max_boosters_for_Sw2.append(boosters_for_Sw2)
                regimes_max_boosters_for_Sw3.append(boosters_for_Sw3)

                regimes_max_boosters.append(
                        boosters_for_R + boosters_for_S + boosters_for_Sf +
                        boosters_for_Sw1 + boosters_for_Sw2 + boosters_for_Sw3)

        max_boosters_for_R.append(regimes_max_boosters_for_R)
        max_boosters_for_S.append(regimes_max_boosters_for_S)
        max_boosters_for_Sf.append(regimes_max_boosters_for_Sf)
        max_boosters_for_Sw1.append(regimes_max_boosters_for_Sw1)
        max_boosters_for_Sw2.append(regimes_max_boosters_for_Sw2)
        max_boosters_for_Sw3.append(regimes_max_boosters_for_Sw3)
        max_boosters.append(regimes_max_boosters)

# Create list of new susceptible_ICs for each vaccination scenario
scenario_susceptibles_IC = []
scenario_boosted = []
scenario_boost_pop_percent = []
scenario_names = []
age_boosting_scenario_order = []

**Scenario 1**: Prioritising those 75+

In [7]:
scenario_names.append('Prioritise 75+')
age_boosting_scenario_order.append((np.arange(16, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 2**: Prioritising those 60-74 then 75+

In [8]:
scenario_names.append('Prioritise 60-74 then 75+')
age_boosting_scenario_order.append([range(12, 15), 15] + (np.arange(12, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 3**: Prioritising those 60-74 then 50-59, then 75+

In [9]:
scenario_names.append('Prioritise those 60-74 then 50-59, then 75+')
age_boosting_scenario_order.append([range(12, 15), range(10, 12), 15] + (np.arange(10, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 4**: Prioritising those 50-59, then 60-74 then 75+

In [10]:
scenario_names.append('Prioritise those 50-59 then 60-74, then 75+')
age_boosting_scenario_order.append([range(10, 12), range(12, 15), 15] + (np.arange(10, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 5**: Prioritising those 20-49, then 50-74 then 75+

In [11]:
scenario_names.append('Prioritise those 20-49, then 50-74 then 75+')
age_boosting_scenario_order.append([range(4, 10), range(10, 15), 15] + (np.arange(4, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 6**: Prioritising those 20-49, then 75+, then 50-74

In [12]:
scenario_names.append('Prioritise those 20-49, then 75+, then 50-74')
age_boosting_scenario_order.append([range(4, 10), 15, range(10, 15)] + (np.arange(4, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 7**: No boosters

In [13]:
scenario_names.append('No boosters')
age_boosting_scenario_order.append((np.arange(16, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0)

### Compute new susceptible conditions based on each boosting scenario

In [14]:
for s, _ in enumerate(scenario_names):
    regimes_new_susceptibles_IC = []
    regimes_boosted = []
    scenario_boosters = (scenario_boost_pop_percent[s] * np.array(total_pop)).tolist()

    for R, _ in enumerate(regimes):
        new_susceptibles_IC = [
            np.array(susceptibles_IC[R][r].copy()).reshape((6, 16)).transpose() for r in range(len(regions))]
        
        new_susceptibles_IC = np.array(new_susceptibles_IC)
        regimes_scenario_boosters = scenario_boosters[R]
        reg_new_susceptibles_IC = []

        boosted = np.zeros_like(new_susceptibles_IC[:, :, 2])

        for r, reg in enumerate(regions):
            for ages in age_boosting_scenario_order[s]:
                if np.sum(max_boosters[R][r][ages]) <= scenario_boosters[R][r]:
                    # Add boosted from the S, Sf, Sw1, Sw2, Sw3 to Sb
                    new_susceptibles_IC[r, ages, 2] += max_boosters[R][r][ages] - max_boosters_for_R[R][r][ages]
                    # Remove boosted from the S
                    new_susceptibles_IC[r, ages, 0] -= max_boosters_for_S[R][r][ages]
                    # Remove boosted from the Sf
                    new_susceptibles_IC[r, ages, 1] -= max_boosters_for_Sf[R][r][ages]
                    # Remove boosted from the Sw1
                    new_susceptibles_IC[r, ages, 3] -= max_boosters_for_Sw1[R][r][ages]
                    # Remove boosted from the Sw2
                    new_susceptibles_IC[r, ages, 4] -= max_boosters_for_Sw2[R][r][ages]
                    # Remove boosted from the Sw3
                    new_susceptibles_IC[r, ages, 5] -= max_boosters_for_Sw3[R][r][ages]
                    # Remove boosted from the S, Sf, Sw1, Sw2, Sw3 and R from total boosters for the scenario
                    scenario_boosters[R][r] -= np.sum(max_boosters[R][r][ages])
                    # Complete boosting vaccination vector
                    boosted[r, ages] = max_boosters[R][r][ages]
                else:
                    # Compute proportion of boosters we have left to give for the age group
                    prop = scenario_boosters[R][r] / np.sum(max_boosters[R][r][ages])
                    # Add boosted from the S, Sf, Sw1, Sw2, Sw3 to Sb
                    new_susceptibles_IC[r, ages, 2] += prop * (max_boosters[R][r][ages] - max_boosters_for_R[R][r][ages])
                    # Remove boosted from the S
                    new_susceptibles_IC[r, ages, 0] -= prop * max_boosters_for_S[R][r][ages]
                    # Remove boosted from the Sf
                    new_susceptibles_IC[r, ages, 1] -= prop * max_boosters_for_Sf[R][r][ages]
                    # Remove boosted from the Sw1
                    new_susceptibles_IC[r, ages, 3] -= prop * max_boosters_for_Sw1[R][r][ages]
                    # Remove boosted from the Sw2
                    new_susceptibles_IC[r, ages, 4] -= prop * max_boosters_for_Sw2[R][r][ages]
                    # Remove boosted from the Sw3
                    new_susceptibles_IC[r, ages, 5] -= prop * max_boosters_for_Sw3[R][r][ages]
                    # Remove boosted from the S, Sf, Sw1, Sw2, Sw3 and R from total boosters for the scenario
                    scenario_boosters[R][r] -= prop * np.sum(max_boosters[R][r][ages])
                    # Complete boosting vaccination vector
                    boosted[r, ages] = prop * max_boosters[R][r][ages]

            reg_new_susceptibles_IC.append(list(deepflatten(new_susceptibles_IC[r].transpose())))

        regimes_new_susceptibles_IC.append(reg_new_susceptibles_IC)
        regimes_boosted.append(boosted)
    scenario_susceptibles_IC.append(regimes_new_susceptibles_IC)
    scenario_boosted.append(regimes_boosted)

### Set the parameters and initial conditions of the model and bundle everything together

In [15]:
# Instantiate model
model = wm.WarwickLancSEIRModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)
model.set_age_groups(age_groups)

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

### Simulate for the regions

In [16]:
# Simulate for all the regions and regimes
outputs = []
total_pop = []

for s, scenario in enumerate(scenario_names):
    scenario_outputs = []
    scenario_total_pop = []
    for R, regime in enumerate(regimes):
        model.read_contact_data(matrices_contact[R], time_changes_contact[R])
        model.read_regional_data(matrices_region[R], time_changes_region[R])

        # Set regional and time dependent parameters
        regional_parameters = wm.RegParameters(
            model=model,
            region_index=1
        )

        # Set ICs parameters
        ICs_parameters = wm.ICs(
            model=model,
            susceptibles_IC=scenario_susceptibles_IC[s][R],
            exposed1_IC=exposed1_IC[R],
            exposed2_IC=exposed2_IC[R],
            exposed3_IC=exposed3_IC[R],
            exposed4_IC=exposed4_IC[R],
            exposed5_IC=exposed5_IC[R],
            infectives_sym_IC=infectives_sym_IC[R],
            infectives_asym_IC=infectives_asym_IC[R],
            recovered_IC=recovered_IC[R]
        )

        # Compute age-dependent population
        scenario_total_pop.append(ICs_parameters.total_population())

        # Set disease-specific parameters
        disease_parameters = wm.DiseaseParameters(
            model=model,
            d=d[R][0],
            tau=tau[R],
            we=we[R],
            omega=omega[R]
        )

        # Set transmission parameters
        transmission_parameters = wm.Transmission(
            model=model,
            beta=beta[R][0],
            alpha=alpha[R],
            gamma=gamma[R]
        )

        # Set other simulation parameters
        simulation_parameters = wm.SimParameters(
            model=model,
            method='Radau',
            times=times,
            eps=False
        )

        # Set vaccination parameters
        vaccine_parameters = wm.VaccineParameters(
            model=model,
            vac=vac,
            vacb=vacb,
            adult=adult,
            nu_tra=nu_tra[R],
            nu_symp=nu_symp[R],
            nu_inf=nu_inf[R],
            nu_sev_h=nu_sev_h[R],
            nu_sev_d=nu_sev_d[R],
        )

        # Set social distancing parameters
        soc_dist_parameters = wm.SocDistParameters(
            model=model,
            phi=1
        )

        # Set all parameters in the controller
        parameters = wm.ParametersController(
            model=model,
            regional_parameters=regional_parameters,
            ICs_parameters=ICs_parameters,
            disease_parameters=disease_parameters,
            transmission_parameters=transmission_parameters,
            simulation_parameters=simulation_parameters,
            vaccine_parameters=vaccine_parameters,
            soc_dist_parameters=soc_dist_parameters
        )

        # Simulate for all the regions
        regimes_outputs = []
        for r, reg in enumerate(regions):
            # List of initial conditions and parameters that characterise the model
            parameters.regional_parameters.region_index = r + 1

            parameters.disease_parameters.d = d[R][r]
            parameters.transmission_parameters.beta = beta[R][r]

            # Simulate using the ODE solver
            regimes_outputs.append(model.simulate(parameters))

        scenario_outputs.append(regimes_outputs)

    outputs.append(scenario_outputs)
    total_pop.append(scenario_total_pop)

outputs = np.array(outputs)

## Plot the comparments of the two methods against each other
### Setup ``plotly`` and default settings for plotting

In [ ]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'orange', 'green', 'gray', 'purple', 'black', 'pink']

## Number of New Infections, Hospitalisations, Deaths & Total number of years of life lost

In [ ]:
# Simulate for all the regions (sum over all age groups)
total_years_of_life_lost = []
total_boosted = []

for s, scenario in enumerate(scenario_names):
    scenario_total_years_of_life_lost = []
    scenario_total_boosted = []
    for R, regime in enumerate(regimes):
        regimes_total_years_of_life_lost = []
        regimes_boosted = []
        for r, reg in enumerate(regions):
            # Compute regional matrix of new infections for all timepoints simulated
            reg_new_infections = model.new_infections(outputs[s, R, r, :, :])

            # Compute regional matrix of new hospitalisation for all timepoints simulated
            reg_new_hospitalisation = model.new_hospitalisations(reg_new_infections, pItoH[R][r], dItoH[R])

            # Compute regional matrix of new deaths for all timepoints simulated
            reg_new_deaths = model.new_deaths(reg_new_hospitalisation, pHtoD[R][r], dHtoD[R])

            regimes_total_years_of_life_lost.append((10**5) / np.sum(total_pop[s][R][r]) * np.sum(np.matmul(
                reg_new_deaths[0] + reg_new_deaths[1] + reg_new_deaths[2] +
                reg_new_deaths[3] + reg_new_deaths[4] + reg_new_deaths[5],
                np.diag(life_ex[r])), axis=1))

            regimes_boosted.append((10**5) / np.sum(total_pop[s][R][r]) * np.array(scenario_boosted[s][R][r]))

        scenario_total_years_of_life_lost.append(regimes_total_years_of_life_lost)
        scenario_total_boosted.append(regimes_boosted)
    
    total_years_of_life_lost.append(scenario_total_years_of_life_lost)
    total_boosted.append(scenario_total_boosted)

total_years_of_life_lost = np.array(total_years_of_life_lost)
total_boosted = np.array(total_boosted)

In [ ]:
# Set up traces to plot
total_years_of_life_lost_mean = []
total_years_of_life_lost_upper = []
total_years_of_life_lost_lower = []

for r, _ in enumerate(regions):
    # Compute the mean 
    total_years_of_life_lost_mean.append(np.mean(total_years_of_life_lost[:,:,r,:], axis=1))

    # Compute the upper quantiles
    total_years_of_life_lost_upper.append(np.quantile(total_years_of_life_lost[:,:,r,:], 0.975, axis=1))

    # Compute the lower qunatiles
    total_years_of_life_lost_lower.append(np.quantile(total_years_of_life_lost[:,:,r,:], 0.025, axis=1))

### Plot the time series of numbers of years of life lost for the different regions

In [ ]:
# Trace names - represent the solver used for the simulation
trace_name = ['{}'.format(scenario) for scenario in scenario_names]

my_max = [5.5, 5, 3.5, 1.25, 1, 1.75, 0.8, 3.5]
best_scenario_num = [[1], [1], [1, 2], [1, 2], [1, 2, 3, 4], [1, 2], [1, 2, 3, 4], [1, 2]]

# Plot for each region
for r, reg in enumerate(regions):
    fig = go.Figure()
    # Plot for each boosting scenario
    for s, scenario in enumerate(scenario_names):
        fig.add_trace(
            go.Scatter(
                y=total_years_of_life_lost_mean[r][s, :],
                x=parameters.simulation_parameters.times,
                mode='lines',
                name=trace_name[s],
                line=dict(
                    color=colours[s])
            )
        )

    for s in best_scenario_num[r]:
        fig.data[s-1].name = '(BEST) ' + trace_name[s-1]

    # Plot for each boosting scenario
    for s, scenario in enumerate(scenario_names):
        fig.add_trace(
            go.Scatter(
                y=total_years_of_life_lost_upper[r][s, :].tolist() + total_years_of_life_lost_lower[r][s, :].tolist()[::-1],
                x=parameters.simulation_parameters.times + parameters.simulation_parameters.times[::-1],
                mode='lines',
                name=trace_name[s],
                fill='toself',
                fillcolor=colours[s],
                line_color=colours[s],
                opacity=0.15,
                showlegend=False
            )
        )

        if r in range(3,7):
            fig.add_trace(
                go.Scatter(
                    y=total_years_of_life_lost_mean[r][s, :],
                    x=parameters.simulation_parameters.times,
                    mode='lines',
                    name=trace_name[s],
                    line=dict(
                        color=colours[s]),
                    xaxis='x2',
                    yaxis='y2',
                    showlegend=False
                )
            )

            fig.add_trace(
                go.Scatter(
                    y=total_years_of_life_lost_upper[r][s, :].tolist() + total_years_of_life_lost_lower[r][s, :].tolist()[::-1],
                    x=parameters.simulation_parameters.times + parameters.simulation_parameters.times[::-1],
                    mode='lines',
                    name=trace_name[s],
                    fill='toself',
                    fillcolor=colours[s],
                    line_color=colours[s],
                    opacity=0.15,
                    xaxis='x2',
                    yaxis='y2',
                    showlegend=False
                )
            )

    # Add axis labels
    fig.update_layout(
        boxmode='group',
        title='Years of Life Lost per 100,000 for {}'.format(reg),
        width=800,
        height=500,
        plot_bgcolor='white',
        xaxis=dict(
            linecolor='black',
            title='Time (days)'),
        yaxis=dict(
            linecolor='black',
            title='Years of Life Lost (per 100k)',
            range=[0, 5.5]),
        xaxis2 = dict(
            linecolor='black',
            domain = [0.25, 0.9],
            anchor = 'y2'
        ),
        yaxis2 = dict(
            showline = True,
            linecolor='black',
            range=[0, my_max[r]],
            domain = [0.5, 0.9],
            anchor = 'x2'
        ),
        hovermode='x unified'
        )

    fig.write_image('images/Years of Life Lost for {}.pdf'.format(reg))
    fig.show()

### Plot the time series of numbers of years lost for different scenarios

In [ ]:
# Trace names - represent the solver used for the simulation
trace_name = ['region {}'.format(r) for r in regions]

# Plot for each boosting scenario
for s, scenario in enumerate(scenario_names):
    fig = go.Figure()
    # Plot (line plot for each solver method for each age)
    for r, reg in enumerate(regions):
        if r % 2 == 1:
            fig.add_trace(
                go.Scatter(
                    y=total_years_of_life_lost_mean[r][s, :],
                    x=parameters.simulation_parameters.times,
                    mode='lines',
                    name=trace_name[r],
                    line=dict(
                        color=colours[int(np.floor(r / 2))],
                        dash='dash')
                )
            )
        else:
            fig.add_trace(
                go.Scatter(
                    y=total_years_of_life_lost_mean[r][s, :],
                    x=parameters.simulation_parameters.times,
                    mode='lines',
                    name=trace_name[r],
                    line=dict(
                        color=colours[int(np.floor(r / 2))])
                )
            )

        fig.add_trace(
            go.Scatter(
                y=total_years_of_life_lost_upper[r][s, :].tolist() + total_years_of_life_lost_lower[r][s, :].tolist()[::-1],
                x=parameters.simulation_parameters.times + parameters.simulation_parameters.times[::-1],
                mode='lines',
                name=trace_name[r],
                fill='toself',
                fillcolor=colours[int(np.floor(r / 2))],
                line_color=colours[int(np.floor(r / 2))],
                opacity=0.15,
                showlegend=False
            )
        )

    # Add axis labels
    fig.update_layout(
        boxmode='group',
        title='Years of Life Lost per 100,000 for Scenario:<br>{}'.format(scenario),
        width=800,
        height=500,
        plot_bgcolor='white',
        xaxis=dict(
            linecolor='black',
            title='Time (days)'),
        yaxis=dict(
            linecolor='black',
            title='Years of Life Lost (per 100k)',
            range=[0, 5.5]),
        hovermode='x unified'
        )

    fig.write_image('images/Years of Life Lost Scenario {}.pdf'.format(s+1))
    fig.show()

## Summary Figure

### Cummulative Years of Life Lost

In [ ]:
new_colours = ['blue', 'orange', 'green', 'red', 'purple', 'gray', 'black']

trace_name = ['{}, {}% boosted'.format(scenario, 100 * scenario_boost_pop_percent[s]) for s, scenario in enumerate(scenario_names)]

fig = go.Figure()
# Plot for each region
for s, scenario in enumerate(scenario_names):
    fig.add_trace(
        go.Bar(
            y=[np.sum(total_years_of_life_lost_mean[r][s, :]) for r, reg in enumerate(regions)],
            x=regions,
            name=scenario_names[s],
            marker_color=new_colours[s],
        )
    )

# Add axis labels
fig.update_layout(
    boxmode='group',
    title='Years of Life Lost per 100,000 for different countries and scenarios',
    width=1000,
    height=500,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    hovermode='x unified',
    legend=dict(
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1.3
        )
    )

fig.write_image('images/Years of Life Lost per Scenario and Country.pdf')
fig.show()